In [26]:
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.freesurfer as fs
from nipype.interfaces.io import DataSink 

In [27]:
bet = pe.MapNode(interface=fsl.BET(), name = 'bet', iterfield=['frac'])
bet.inputs.in_file = 'ms1297-mse4664-007-MPRAGE_iso_1_pve_2.nii.gz'
bet.inputs.frac = [0.7, 0.5, 0.3]
#can now adjust to process through different fracs
fast = pe.MapNode(interface=fsl.FAST(), name='fast', iterfield=['in_files'])
ss = pe.MapNode(interface = fs.SegStats(),name='ss', iterfield=['in_files'])
#iterate over a list of fast outputs as a mapnode, iterfield = in_files  
#ds = pe.Node(interface=)

In [28]:
workflow = pe.Workflow(name='MapNodeFlow')
workflow.base_dir = '.'

In [29]:
workflow.connect([(bet, fast, [('out_file', 'in_files')]), (fast, ss,[('mixeltype', 'in_files')])])
#for FAST use mixeltypes instead of out_files 
#datasink works the same but you can invent the input and it'll create a file though 

In [30]:
workflow.run() 
#check the JSONS for t1 files and know that eventually the UTE files will have another field called UTE 
#less status.JSON
#check nii/status.JSON

RuntimeError: Workflow did not execute cleanly. Check log for details